# Proyek Pertama ML Terapan
- **Nama:** Lusi Aulia Jati
- **Email:** lusiauliajati@gmail.com
- **Sumber Data:** https://www.kaggle.com/datasets/l3llff/banana

###  Data Loading

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns

# load the dataset
bnn = pd.read_csv("./banana_quality.csv")
bnn.head()

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290,Good
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325,Good
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322,Good
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862,Good
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442,Good


In [ ]:
#melihat informasi jumlah dan jenis data 
bnn.info()

In [ ]:
#melihat statistik deskriptif dari data
bnn.describe()

### Exploratory Data Analysis

In [ ]:
#mengecek apakah ada data kosong
bnn.isnull().sum()

In [ ]:
#outlier dari data
sns.boxplot(bnn)

In [ ]:
Q1 = bnn.quantile(0.25)
Q3 = bnn.quantile(0.75)
IQR=Q3-Q1
bnn=bnn[~((bnn<(Q1-1.5*IQR))|(bnn>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah drop outliers
bnn.shape

In [ ]:
sns.boxplot(bnn)

sudah bersih dari outlier

In [ ]:
plt.figure()
correlation_matrix = bnn.corr().round(2)
 
# Untuk menge-print nilai di dalam kotak, gunakan parameter anot=True
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, )
plt.title("Correlation Matrix untuk Fitur Numerik ", size=20)

In [ ]:
g = sns.pairplot(bnn)

Kita akan mneggunakan data waktu panen dan ukuran, apakah waktu panen berpengaruh pada ukuran buah pisang yang dipanen. Menggunakan model K-Nearest Neighbor (KNN), Random Forest, dan Boosting Algorithm. Akan digunakan data train 80% dan test 20%.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

bnn = bnn.drop('Quality',)

scaler = MinMaxScaler()
bnn_scaled = scaler.fit_transform(bnn)
 
X = bnn_scaled["HarvestTime"]
y = bnn_scaled["Size"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [ ]:
print(f'Total # of sample in whole dataset: {len(X)}')
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Total # of sample in test dataset: {len(X_test)}')

In [ ]:
scaler = MinMaxScaler()
bnn_scaled = scaler.fit_transform(df)

total_dataset = len(df)
df_train = df_scaled[:int(total_dataset*0.80)]
df_test = df_scaled[int(total_dataset*0.80):total_dataset]

print('Data for train:', df_train.shape)
print('Total day for train:', round(df_train.shape[0]/24))
print('\nData for test:', df_test.shape)
print('Total day for test:', round(df_test.shape[0]/24))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
 
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train, y_train)
 
models.loc['train_mse','knn'] = mean_squared_error(y_pred = knn.predict(X_train), y_true=y_train)

In [ ]:
# Visualising a Subset of our data - important features

g = sns.pairplot(raw_data[['Visitors', 'Revenue', 'Marketing Spend', 'Day_Name']], hue = 'Day_Name', height = 5)

In [ ]:
# Visualising a Subset of our data - important features

g = sns.lmplot(x = 'Marketing Spend', y = 'Revenue', data = raw_data, col = 'Day_Name', col_wrap = 3, height = 5, 
              scatter_kws = {'color':'green'}, ci = False)

In [ ]:
# Investigate the distribution of Revenue by Categorical Variables

pal = ['green', 'blue','red']

g = sns.boxplot(x = 'Day_Name', y = 'Revenue', data = raw_data, hue = 'Promo', palette = pal)
               
#ax = sns.swarmplot(x = 'Day_Name', y = 'Revenue', data = raw_data, palette = pal, hue = 'Promo')

### 5.2. Data Cleaning

In [ ]:
# Deleting outliers

raw_data = raw_data[raw_data['Revenue'] < 27000]

raw_data.shape

In [ ]:
# Column names
raw_data.columns

In [ ]:
# Selecting usefull columns only
raw_data = raw_data[['Day_Name','Visitors', 'Revenue', 'Marketing Spend', 'Promo']]

#visualize the raw data
raw_data.head()

In [ ]:
# Making categorical variables into numeric representation

new_raw_data = pd.get_dummies(raw_data, columns = ['Promo', 'Day_Name'])

# Notes:
# We can also do this with Label Encoding and OneHotEncoder from the preprocessing library

# Visualizing the data
new_raw_data.head()

### 5.3. Feature Selection

In this example, we do not have many variables so we should use all of the data but in some cases, you have thousands of variables and you will need to filter them in order to save computational time

- 2 ways to help us select the important features are:
    - Correlation 
    - Feature Importance

In [ ]:
# Running Correlation

hm = new_raw_data[['Visitors','Revenue','Marketing Spend']].corr()

#### Notes
- .corr() is used to find the pairwise correlation of all columns in the dataframe. Any null values are automatically excluded
- The closer to 1 or -1 the better. As one variable increases, the other variable tends to also increase / decrease
- 0.8 +- is Strong Correlation, 0.6 to 0.8 +- is moderate Correlation & the other values, there is no correlation
- More Info here: https://statisticsbyjim.com/basics/correlations/

In [ ]:
# Visualizing Correlation with a Heatmap

g = sns.heatmap(hm, annot = True, annot_kws={'size':50})

#### Steps of Running Feature Importance
- Split the data into X & y
- Run a Tree-based estimators (i.e. decision trees & random forests) 
- Run Feature Importance

In [ ]:
# Split the data into X & y

X = new_raw_data.drop('Revenue', axis = 1).values
X2 = new_raw_data.drop('Revenue', axis = 1)
y = new_raw_data['Revenue']

print(X.shape)
print(y.shape)

X

In [ ]:
# Run a Tree-based estimators (i.e. decision trees & random forests)

dt = DecisionTreeClassifier(random_state=15, criterion  = 'entropy', max_depth = 10)
dt.fit(X,y)

# If you want to learn how Decesion Trees work, read here: https://www.datacamp.com/community/tutorials/decision-tree-classification-python
# Official Doc: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

#### Notes:
- The importance of a feature is calculated as the (normalized) total reduction of entropy (other criterions too) brought by that feature or the higher information gain
- To understand the maths, read this: https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3

In [ ]:
# Running Feature Importance

for i, column in enumerate(new_raw_data.drop('Revenue', axis = 1)):
    print('The feature importance for {} is: {:.3f}'.format(column, dt.feature_importances_[i]))

#### Notes:
- Please note that we have not normalised / scale our data
- Please note that we have not done any feature engineering - created new features
- Please note that we have not joined multiple datasets together
- Please note that we have not aggregated any of our data

## 6. Splitting the Raw Data - Hold-out validation

In [ ]:
# Hold-out validation

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size = 0.2, random_state=15)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Official Doc: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## 7. What is Linear Regression Analysis

- Linear regression is a widely used machine learning model that predicts continues numbers as outputs; not classes 
- It models the relationship of 2 plus variables by fitting a linear equation to observed data 
- In linear regression you have 1 dependent variable (y) and 1 plus independent variables (X)
- Before you attempt to model the data, you should check if there is a relationship between the variables first; a good way is to use a scatterplot to visualise the data



In [ ]:
### Graphical Anscombe's_quartet

PATH = "F:\\Github\\Python tutorials\\Introduction to ML - LR model\\"
Image(filename = PATH + "Anscombe's_quartet.png", width=900, height=900)


# identical descriptive statistics
# identical mean, std, variances, correlations, and regression lines

### What is Linear Regression Analysis (continued)

- A measurement of the relationship between 2 variables is the correlation coefficient; ranging from -1 to 1; the closer to 1 or -1, the stronger the relationship

- However, even if there seems to be a relationship, you should always be careful when modelling the relationship between 2 variables as correlation is not causation. For example, the rise of temperature vs Revenue

- The mathematical equation of linear regression is Y = a + bX; where X is the independent variable and Y is the dependent variable. ‘b’ is the slope of the line and 'a' is the intercept; the value of y when x = 0


- The most common cost function used in linear regression is the “Least Squared Errors” function; which is the sum of squared errors (sum(y actuals – y predicted) ^ 2) over the training set; trying to minimize how far off the predictions are from the actuals.

- To calculate the “Least Squared Errors” firstly you must calculate the difference of the actual (y) vs the p(y); this is called the residuals.

- In order to find the least squared error you will have to find the optimal parameter values (b) that minimize the sum ‘S’ of squared residuals


In [ ]:
# How L Regression iterates until convergence

PATH = "F:\\Github\\Python tutorials\\Introduction to ML - LR model\\"
Image(filename = PATH + "Regression Example.png", width=900, height=900)


#### What is Linear Regression Analysis (continued)

- The optimisation technique used in linear regression is Gradient Descent; which attempts to find a local or a global minimum of a cost function

- Gradient descent finds the direction ‘gradient’ that the model/line should take so that the errors will be reduced. 

- Direction refers to the weights (a and b), and how those weights should be tweaked to further reduce the errors. 

- The model will iterate until converged; no further improvements can be done; see graph below: 

In [ ]:
# Gradient Descent

PATH = "F:\\Github\\Python tutorials\\Introduction to ML - LR model\\"
Image(filename = PATH + "Gradient Descent.png", width=900, height=900)

## 8. Running Linear Regression

In [ ]:
# Training the Regression

lm = LinearRegression(fit_intercept = True)
lm.fit(X_train, y_train)

y_pred = lm.predict(X_train)
# SK-Learn official doc: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

## 9. Evaluating the Model

In [ ]:
# Model Accuracy on training dataset

print('The Accuracy  on the training dataset is: ', lm.score(X_train, y_train) )
print('The Accuracy n2  on the training dataset is: ',r2_score(y_train,y_pred) )   

print("")
# Model Accuracy on testing dataset
print('The Accuracy  on the testing dataset is: ', lm.score(X_test, y_test) )

print("")
# The Root Mean Squared Error (RMSE)
print('The RMSE  on the training dataset is: ',sqrt(mean_squared_error(y_train,y_pred)))
print('The RMSE  on the testing dataset is: ',sqrt(mean_squared_error(y_test,lm.predict(X_test))))

print("")
# The Mean Absolute Error (MAE)
print('The MAE  on the training dataset is: ',mean_absolute_error(y_train,y_pred))
print('The MAE  on the testing dataset is: ',mean_absolute_error(y_test,lm.predict(X_test)))


print("")
# Coefficients
print('Coefficients: ', lm.coef_ )

print("")
# The Intercept
print('Intercept: ', lm.intercept_)


# R2 Link: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# RMSE Link: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
# MAE Link: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

#### Notes:
    
- The accuracy score represents the coefficient of determination ( 𝑅2 ). This is at max 1, but can be negative. It will be 0 if you predict the mean of y for all observations.

- The R Square is measure of how close the data are to the fitted regression line. 

- In this case we can say that our model explains 79% of the training data & 75% of the testing data


- The RMSE is the standard deviation of the residuals. Residuals is the difference between the predicted value and the  regression line. Hence RMSE is a measure of how spread your residuals are.

- The mean absolute error (MAE) is the average of all the absolute errors. The absolute error is the difference between the true value (y_train) and the predicted value (y_pred).

- Coeff are the weights

- The intercept is the expected mean value of Y when all X=0

In [ ]:
# Plotting Actuals Vs Predicted

plt.figure(figsize=(15,10))

plt.scatter(y_train, y_pred, c='green')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', c='red', lw=3)
plt.xlabel('Actuals')
plt.ylabel('Predicted Values')
plt.title('Actuals Vs Predicted Values')
# increase size



In [ ]:
# Ploting Residuals

plt.figure(figsize=(15,10))

sns.residplot(y_train, y_pred, color='green')
plt.xlabel('Actual Revenue')
plt.ylabel('Residuals')
plt.title('Actuals Vs Residuals')



In [ ]:
X2.columns

##### The next step should be to go back, remove more outliers and check if our model can be improved

## 10. How to use our L. Regression model to Make Predictions

In [ ]:
# Passing Coeff into a table

Coeff = lm.coef_
Coeff.shape

# Reshaping
Coeff = Coeff.reshape(-1,12)


# Creating a Dataframe
Coeff_df = pd.DataFrame(Coeff, columns = [X2.columns])

# Displaying 
Coeff_df
